In [1]:
import json

In [2]:
import sys
sys.path.append('../library')

from helpers import *

In [64]:
###
### Concatenteaza o lista de string.
### param list_: lista de string.
### return string-ul rezultat dupa concatenare.
###
def get_key_from_lists(list_):
    key = ""
    for elem in list_:
        key += elem + ' '
    return key[0:len(key)-1]

###
### Calculeaza distanta euclidiana dintre un vector si o lista de vectori si returneaza o lista cu cei mai aporpiati nr.
### param X: dictionar {cheie:vector}. -> vectorul
### param X_dict: dictionar {chei:vectori} -> vectorii.
### param nr: numarul de vecini cei mai apropiati pe care ii va returna.
### return distante: dictionar {chei:vectori}.
### 
def calculeaza_distante_p(X, X_dict, nr):
    distante = {}
    for elem in X_dict:
        distante[elem] = distance.euclidean(X, X_dict[elem])        
    distante = sort_dict_value(distante)
    return distante[0:nr]

def get_fos_name(fos):
    list_fos = []
    for elem in fos:
        list_fos.append(elem['name'])
    return list_fos

###
### Parcurge toate fisierele si creeaza un dictionar {id:[list_of_fos]}.
### file_path: calea spre fisiere.
### return dict_: dictionarul.
###
def get_dict_foses(file_path):
    dict_ = {}
    
    for i in range(0,1):
        file = file_path + str(i) + '.txt'
        r = open(file,'r')
        
        for linie in r:
            articol_crt = json.loads(linie)
            id_ = articol_crt['id']
            fos = articol_crt['fos']
            dict_[id_] = get_fos_name(fos)
        r.close()
        
    return dict_



###
### Parcurge toate elementele unui dictionar de tipul {abstract:[encoding]}
### param dict_1: dictionarul.
### param vecini: numarul de vecini cautati.
### return dict_2: dictionar ce contine vecini nr de vecini [abstract:[abstract]] 
### 
def get_dict_similar(dict_1, vecini):
    dict_2 = {}
    k = 0
    j = 0
    for key in dict_1:
        if j % 1000 == 0:
            print(j)
        j+=1
        new_X_dict = elimina_elem_dupa_cheie(dict_1, key)
        distante = calculeaza_distante_p(dict_1[key],new_X_dict, vecini) 
        list_ = []
        for elem in distante:
            list_.append(elem[0])

        dict_2[key] = list_
        
    return dict_2
    

In [44]:
def compute_union(d1, d2):
    union = []
    union += d1
    union += d2
    union = remove_duplicates(union)
    return len(union)

def jaccard(dict_id_fos, dict_id_vecini):   
    procent_total = 0
    k = 0
    valids = 0
    i=1
    for elem in dict_id_fos:
        i+=1
        l2 = []
        l1 = dict_id_fos[elem] # toate fos pt venue curent
        
        try:
            for fos in dict_id_vecini[elem]: # caut fos de la vecini
                l2 += dict_id_fos[fos]
        except KeyError:
            continue
        l2 = remove_duplicates(l2)
        valids += 1
        
        # Gasesc fos din venue curent care se gasesc si la vecini
        l_comun = elemente_comune_lista(l1,l2)
        
        # jacard pentru A = fos care se gasesc si la vecini, B = fos pentru venue curent
        proc = len(l_comun)/len(l1)
       
        procent_total += proc        
    
    procent_total /= valids
    
    return procent_total


In [60]:
def get_model_vectors(file, field = ""):
    X_dict = {}
    if field == "":
        field = "abstract"
    r = open(file+'0.txt','r')
    for line in r:
        art_crt = json.loads(line)
        X_dict[art_crt['id']] = art_crt[field]
    r.close()
    return X_dict

In [16]:
dict_abstract_fos = get_dict_foses('../Date/Initiale/Subset/file')
print(len(dict_abstract_fos))

10000


## Word2vec gensim avg

In [20]:
X_dict = get_model_vectors('../Date/word2vec-gensim/Abstract/Subset/avg/file')
print(len(X_dict))

10000


In [ ]:
# dict cu {id:[similar_id]}
abstr = get_dict_similar(X_dict,20)
print(len(abstr))

In [31]:
j = jaccard(dict_abstract_fos,abstr)
print(j) # 20

i 10001
0.6098367954268024


In [36]:
for elem in abstr:
    abstr[elem] = abstr[elem][0:15]

In [37]:
j = jaccard(dict_abstract_fos,abstr)
print(j) # 15

i 10001
0.5801419000444047


In [38]:
for elem in abstr:
    abstr[elem] = abstr[elem][0:10]

In [39]:
j = jaccard(dict_abstract_fos,abstr)
print(j) # 10

i 10001
0.5381475660450666


In [40]:
for elem in abstr:
    abstr[elem] = abstr[elem][0:5]

In [41]:
j = jaccard(dict_abstract_fos,abstr)
print(j) # 5

i 10001
0.4622594560994581


## Word2vec gensim sum

In [42]:
X_dict = get_model_vectors('../Date/word2vec-gensim/Abstract/Subset/sum/file')

In [43]:
# dict cu {id:[similar_id]}
abstr = get_dict_similar(X_dict,20)
print(len(abstr))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [45]:
# 20
j = jaccard(dict_abstract_fos,abstr)
print(j,'20')

# 15
for elem in abstr:
    abstr[elem] = abstr[elem][0:15]
j = jaccard(dict_abstract_fos,abstr)
print(j,'15') 

#10
for elem in abstr:
    abstr[elem] = abstr[elem][0:10]
j = jaccard(dict_abstract_fos,abstr)
print(j,'10') 

# 5
for elem in abstr:
    abstr[elem] = abstr[elem][0:5]
j = jaccard(dict_abstract_fos,abstr)
print(j,'5') 

0.5827623967699009 20
0.5553515861915909 15
0.5148918126318156 10
0.4434783974359014 5


## Word2vec pretrained avg

In [47]:
X_dict = get_model_vectors('../Date/word2vec-pretrained/Abstract/Subset/avg/file')

# dict cu {id:[similar_id]}
abstr = get_dict_similar(X_dict,20)
print(len(abstr))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [48]:
# 20
j = jaccard(dict_abstract_fos,abstr)
print(j,'20')

# 15
for elem in abstr:
    abstr[elem] = abstr[elem][0:15]
j = jaccard(dict_abstract_fos,abstr)
print(j,'15') 

#10
for elem in abstr:
    abstr[elem] = abstr[elem][0:10]
j = jaccard(dict_abstract_fos,abstr)
print(j,'10') 

# 5
for elem in abstr:
    abstr[elem] = abstr[elem][0:5]
j = jaccard(dict_abstract_fos,abstr)
print(j,'5') 

0.5527322649572669 20
0.522492796092797 15
0.47968737234987285 10
0.4082762942613008 5


## Word2vec pretrained sum

In [50]:
X_dict = get_model_vectors('../Date/word2vec-pretrained/Abstract/Subset/sum/file')

# dict cu {id:[similar_id]}
abstr = get_dict_similar(X_dict,20)
print(len(abstr))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [51]:
# 20
j = jaccard(dict_abstract_fos,abstr)
print(j,'20')

# 15
for elem in abstr:
    abstr[elem] = abstr[elem][0:15]
j = jaccard(dict_abstract_fos,abstr)
print(j,'15') 

#10
for elem in abstr:
    abstr[elem] = abstr[elem][0:10]
j = jaccard(dict_abstract_fos,abstr)
print(j,'10') 

# 5
for elem in abstr:
    abstr[elem] = abstr[elem][0:5]
j = jaccard(dict_abstract_fos,abstr)
print(j,'5') 

0.5035644533244549 20
0.4758358055833061 15
0.4364454742479769 10
0.37092325008325516 5


## Doc2vec

In [65]:
X_dict = get_model_vectors('../Date/doc2vec/Abstract/Subset/file', "d2w")

# dict cu {venue:[similar_venues]}
abstr = get_dict_similar(X_dict,20)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [66]:
# 20
j = jaccard(dict_abstract_fos,abstr)
print(j,'20')

# 15
for elem in abstr:
    abstr[elem] = abstr[elem][0:15]
j = jaccard(dict_abstract_fos,abstr)
print(j,'15') 

#10
for elem in abstr:
    abstr[elem] = abstr[elem][0:10]
j = jaccard(dict_abstract_fos,abstr)
print(j,'10') 

# 5
for elem in abstr:
    abstr[elem] = abstr[elem][0:5]
j = jaccard(dict_abstract_fos,abstr)
print(j,'5') 

0.4386582295482338 20
0.4138537973138025 15
0.3797151975802031 10
0.3205288289488304 5


In [ ]:
venue_neig = {'a':['ana','mere'], 'b':['mere','si','pere']}
venue_fos = {'a':['1','2'],'ana':['2','3'], 'are':['4','1'], 'mere':['5', '6'], 'b':['2','5','1'], 'si':['6'],'pere':['6','2','3']}

In [ ]:
a : [1 2] [2 3 5] i: [2] u: [1 2 3 4 5] r: 1/5
b : [1 2 5 6 ] [2 3 5 6] i: [2 5 6] u: [1 2 3 5 6] r: 3/5